In [2]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [3]:
# Load environment variables in a file called .env

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [4]:
openai = OpenAI()

In [5]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [6]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."},
        {"role": "user", "content": user_prompt_for(website)}
    ]

# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt
    
# Generate a summary of content fetched by scraping the website

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [8]:
# Step 1: Create your prompts
WEBSITE_LINK = "https://www.growthschool.io/in"

system_prompt = "You are an assistant that analyzes the contents of an event \
and provides short summaries for a Twitter post, an instagram post and a facebook post.\
Ensure the summaries abide by the platform rules for each of the platforms."

website_summary = summarize(WEBSITE_LINK)
user_prompt = f"The events details are as follows: {website_summary}. Please summarize the above. Capture details like time and location, please capture them as well."

# Step 2: Make the messages list

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt},
]

# Step 3: Call OpenAI

response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)

# Step 4: print the result

display(Markdown(response.choices[0].message.content))

**Twitter Post:**
🚀 Join GrowthSchool to transform into the top 1%! 🎓 Our certified courses, expert mentorship, and vibrant community empower you in Product, Growth, Design, Tech & Data. 🌟 Network with 1,000+ learners & gain skills for an AI-driven world. Future-proof your career today! #GrowthSchool

**Instagram Post:**
🌟Become part of the elite 1% with GrowthSchool!✨ Dive into our certified courses across Product Management, Digital Marketing, UI/UX Design, and more! Connect with a thriving community, learn from industry experts, and tackle real-world challenges! 🚀 Join us on this transformative journey! #GrowthSchool #CareerGrowth

**Facebook Post:**
💡 Unlock your potential with GrowthSchool! Our platform offers certified courses in high-demand fields like Digital Marketing, Cloud Data Engineering, and UI/UX Design. Join a community of 1,000+ learners, enjoy hands-on projects, and benefit from mentorship with industry leaders. 🌍 Sign up now to thrive in your career and become a part of the top 1%!🔥 Don't miss out on exciting events and workshops—stay updated with us! #GrowthSchool #ProfessionalDevelopment